In [1]:
import pandas as pd

In [8]:
# reading the data
ruleset = pd.read_excel('Example_firewall_rules.xlsx', sheetname=['rules'])
# print(ruleset)
ruleset = ruleset['rules'] # get df

In [9]:
ruleset.head()

,Unnamed: 0,FW,#,Source,Destination,Service,Action,Track,Install On,Time,Comment
0,NaN,FW-1-2,1.0,global-Mgt,Any,administration-ports,accept,None,Gateways,Any,NaN
1,NaN,FW-1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,FW-1-2,2.0,gobal-Finance,Any,accounting-ports,accept,Log,Gateways,Any,Financial operations and deployment;access via...
3,NaN,FW-1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,FW-1-2,3.0,global-NationX,Amy,block-NationX,deny,Log,Gateways,Any,deny access of nationX to our systems


In [ ]:
# <do the same for hostgroup objects>

In [32]:
# identify rules, each new rules is indicated by a nr.
ruleset['#'].dropna().index


Int64Index([0, 2, 4, 6, 9, 12, 15, 19, 23, 31, 34], dtype='int64')

In [ ]:
# loop over the rows of the dataframe, collecting data of Source, Destination, Service and Action 
# between unique row identifiers and store them in a single row of a new dataframe

# Next: write additional logic to exclude 'noise' in the Source column (Deployment and TheRealBaddie arent't part
# of any rule). You cannot drop them based on name, real life firewall files are over 2000 rules

In [31]:
# Below you will find the code to wrangle the original ruleset (real firewall file, 
# not guaranteed to work when copy-pasted)

In [ ]:
rule_idx = ruleset['#'].dropna().index

def wrangle_df(df_in, rule_idx):
    df_out = pd.DataFrame(columns=df_in.columns)
    for idx in rule_idx:
        row = df_in.iloc[idx].copy()
        source = row['Source']
        dest = row['Destination']
        service = row['Service']
        same_rule = True
        src_was_nan = False
        counter = idx + 1
        while same_rule:
            try:
                rule = df_in.iloc[counter].copy()
            except IndexError:  # end of file
                same_rule = False
                continue
            rule_src = rule['Source']
            rule_dst = rule['Destination']
            rule_srvc = rule['Service']
            rule_nr = rule['#']
#             print(rule_nr)
            #             if (rule_src == 'nan') & (rule_dst == 'nan'):  # end of rule when both are NA
            if not rule_nr == 'nan':
                same_rule = False
                if type(source) is list:
                    row['Source'] = ','.join(source)
                if type(dest) is list:
                    row['Destination'] = ','.join(dest)
#                 if not (rule_srvc == 'nan'):  # check if service has one more entry before setting value
                if type(service) is list:
                    row['Service'] = ','.join(service)
                continue
            else:
                if not (rule_src == 'nan'): # if value not nan, append
                    if src_was_nan:
                        pass
                    elif type(source) is not list:
                        source = [source]
                        source.append(rule_src)
                    else: source.append(rule_src)
                else: # source can be filled with meta data before new rule, we don't want this info
                    src_was_nan = True
                if not (rule_dst == 'nan'):
                    if type(dest) is not list:
                        dest = [dest]
                    dest.append(rule_dst)
                if not (rule_srvc == 'nan'):
                    if type(service) is not list:
                        service = [service]
                    service.append(rule_srvc)
            counter += 1
        # add row to new df
        df_out = df_out.append(row)
    df_out.reset_index(drop=True, inplace=True)
    return df_out

In [ ]:
# clean_rules = wrangle_df(ruleset, rule_idx)